In [52]:
import pandas as pd
data = pd.read_csv(r"D:\ches game\chess_games.csv")
data.head()


,game_id,rated,turns,victory_status,winner,time_increment,white_id,white_rating,black_id,black_rating,moves,opening_code,opening_moves,opening_fullname,opening_shortname,opening_response,opening_variation
0,1,False,13,Out of Time,White,15+2,bourgris,1500,a-00,1191,d4 d5 c4 c6 cxd5 e6 dxe6 fxe6 Nf3 Bb4+ Nc3 Ba5...,D10,5,Slav Defense: Exchange Variation,Slav Defense,NaN,Exchange Variation
1,2,True,16,Resign,Black,5+10,a-00,1322,skinnerua,1261,d4 Nc6 e4 e5 f4 f6 dxe5 fxe5 fxe5 Nxe5 Qd4 Nc6...,B00,4,Nimzowitsch Defense: Kennedy Variation,Nimzowitsch Defense,NaN,Kennedy Variation
2,3,True,61,Mate,White,5+10,ischia,1496,a-00,1500,e4 e5 d3 d6 Be3 c6 Be2 b5 Nd2 a5 a4 c5 axb5 Nc...,C20,3,King's Pawn Game: Leonardis Variation,King's Pawn Game,NaN,Leonardis Variation
3,4,True,61,Mate,White,20+0,daniamurashov,1439,adivanov2009,1454,d4 d5 Nf3 Bf5 Nc3 Nf6 Bf4 Ng4 e3 Nc6 Be2 Qd7 O...,D02,3,Queen's Pawn Game: Zukertort Variation,Queen's Pawn Game,NaN,Zukertort Variation
4,5,True,95,Mate,White,30+3,nik221107,1523,adivanov2009,1469,e4 e5 Nf3 d6 d4 Nc6 d5 Nb4 a3 Na6 Nc3 Be7 b4 N...,C41,5,Philidor Defense,Philidor Defense,NaN,NaN


In [53]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20058 entries, 0 to 20057
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   game_id            20058 non-null  int64 
 1   rated              20058 non-null  bool  
 2   turns              20058 non-null  int64 
 3   victory_status     20058 non-null  object
 4   winner             20058 non-null  object
 5   time_increment     20058 non-null  object
 6   white_id           20058 non-null  object
 7   white_rating       20058 non-null  int64 
 8   black_id           20058 non-null  object
 9   black_rating       20058 non-null  int64 
 10  moves              20058 non-null  object
 11  opening_code       20058 non-null  object
 12  opening_moves      20058 non-null  int64 
 13  opening_fullname   20058 non-null  object
 14  opening_shortname  20058 non-null  object
 15  opening_response   1207 non-null   object
 16  opening_variation  14398 non-null  objec

In [54]:
data.columns

Index(['game_id', 'rated', 'turns', 'victory_status', 'winner',
       'time_increment', 'white_id', 'white_rating', 'black_id',
       'black_rating', 'moves', 'opening_code', 'opening_moves',
       'opening_fullname', 'opening_shortname', 'opening_response',
       'opening_variation'],
      dtype='object')

In [55]:
data.describe()

,game_id,turns,white_rating,black_rating,opening_moves
count,20058.000000,20058.000000,20058.000000,20058.000000,20058.000000
mean,10029.500000,60.465999,1596.631868,1588.831987,4.816981
std,5790.390185,33.570585,291.253376,291.036126,2.797152
min,1.000000,1.000000,784.000000,789.000000,1.000000
25%,5015.250000,37.000000,1398.000000,1391.000000,3.000000
50%,10029.500000,55.000000,1567.000000,1562.000000,4.000000
75%,15043.750000,79.000000,1793.000000,1784.000000,6.000000
max,20058.000000,349.000000,2700.000000,2723.000000,28.000000


In [56]:
# Check missing values
data.isnull().sum()


game_id                  0
rated                    0
turns                    0
victory_status           0
winner                   0
time_increment           0
white_id                 0
white_rating             0
black_id                 0
black_rating             0
moves                    0
opening_code             0
opening_moves            0
opening_fullname         0
opening_shortname        0
opening_response     18851
opening_variation     5660
dtype: int64

In [57]:
data['opening_response'] = data.get('opening_response').fillna('Unknown')
data['opening_variation'] = data.get('opening_variation').fillna('Unknown')



In [58]:
data.duplicated().sum()

np.int64(0)

In [59]:
# التأكد من أنواع البيانات وتحويلها إن لزم
data['rated'] = data['rated'].astype(bool)
data['turns'] = pd.to_numeric(data['turns'], errors='coerce').fillna(0).astype(int)
data['white_rating'] = pd.to_numeric(data['white_rating'], errors='coerce').fillna(0).astype(int)
data['black_rating'] = pd.to_numeric(data['black_rating'], errors='coerce').fillna(0).astype(int)
data['opening_moves'] = pd.to_numeric(data.get('opening_moves', 0), errors='coerce').fillna(0).astype(int)


In [60]:
# تقسيم الوقت بالشكل "5+10" إلى جزئين base_time و increment
def parse_time_increment(x):
    try:
        base, inc = str(x).split('+')
        return int(base), int(inc)
    except Exception:
        return np.nan, np.nan

time_parsed = data['time_increment'].astype(str).apply(parse_time_increment)
data['base_time'] = time_parsed.apply(lambda t: t[0]).fillna(0).astype(int)
data['increment'] = time_parsed.apply(lambda t: t[1]).fillna(0).astype(int)
data['base_time_seconds'] = data['base_time'] * 60


In [61]:
# فرق التقييم بين الأبيض والأسود
data['rating_diff'] = data['white_rating'] - data['black_rating']

# تحويل النتيجة إلى أرقام
data['result'] = data['winner'].map({'White': 1, 'Black': 0})
data['result'] = data['result'].fillna(2).astype(int)

# إنشاء عمود time_control
data['time_control'] = data['base_time'].astype(str) + '+' + data['increment'].astype(str)

# تصنيف تقييمات اللاعبين في فئات (Buckets)
bins = [0, 1000, 1400, 1600, 1800, 2000, 3000]
labels = ['<1000','1000-1399','1400-1599','1600-1799','1800-1999','2000+']
data['white_rating_bucket'] = pd.cut(data['white_rating'], bins=bins, labels=labels, right=False)
data['black_rating_bucket'] = pd.cut(data['black_rating'], bins=bins, labels=labels, right=False)


In [62]:
# نسب الفوز لكل لون
winrates = data['winner'].value_counts(normalize=True).rename_axis('winner').reset_index(name='proportion')
print("Win rates:\n", winrates)

# أشهر 10 افتتاحيات
top_openings = data['opening_fullname'].value_counts().reset_index().rename(columns={'index':'opening_fullname', 'opening_fullname':'count'}).head(10)
print("Top openings:\n", top_openings)

# متوسط عدد النقلات حسب الفائز
avg_turns_by_result = data.groupby('winner')['turns'].mean().reset_index().rename(columns={'turns':'avg_turns'})
print("Avg turns by winner:\n", avg_turns_by_result)


Win rates:
   winner  proportion
0  White    0.498604
1  Black    0.454033
2   Draw    0.047363
Top openings:
                                            count  count
0                           Van't Kruijs Opening    368
1                               Sicilian Defense    358
2               Sicilian Defense: Bowdler Attack    296
3               French Defense: Knight Variation    271
4                                    Scotch Game    271
5  Scandinavian Defense: Mieses-Kotroc Variation    259
6                Queen's Pawn Game: Mason Attack    232
7          Queen's Pawn Game: Chigorin Variation    229
8                           Scandinavian Defense    223
9                                Horwitz Defense    209
Avg turns by winner:
   winner  avg_turns
0  Black  60.724388
1   Draw  86.157895
2  White  57.790221


In [63]:
cols_keep = [
    'game_id','rated','turns','victory_status','winner','result','time_control',
    'base_time','increment','base_time_seconds',
    'white_id','white_rating','white_rating_bucket',
    'black_id','black_rating','black_rating_bucket',
    'rating_diff','opening_fullname','opening_shortname','opening_moves','opening_variation'
]

cols_keep = [c for c in cols_keep if c in data.columns]
clean_data = data[cols_keep].copy()

clean_data.head()


,game_id,rated,turns,victory_status,winner,result,time_control,base_time,increment,base_time_seconds,...,white_rating,white_rating_bucket,black_id,black_rating,black_rating_bucket,rating_diff,opening_fullname,opening_shortname,opening_moves,opening_variation
0,1,False,13,Out of Time,White,1,15+2,15,2,900,...,1500,1400-1599,a-00,1191,1000-1399,309,Slav Defense: Exchange Variation,Slav Defense,5,Exchange Variation
1,2,True,16,Resign,Black,0,5+10,5,10,300,...,1322,1000-1399,skinnerua,1261,1000-1399,61,Nimzowitsch Defense: Kennedy Variation,Nimzowitsch Defense,4,Kennedy Variation
2,3,True,61,Mate,White,1,5+10,5,10,300,...,1496,1400-1599,a-00,1500,1400-1599,-4,King's Pawn Game: Leonardis Variation,King's Pawn Game,3,Leonardis Variation
3,4,True,61,Mate,White,1,20+0,20,0,1200,...,1439,1400-1599,adivanov2009,1454,1400-1599,-15,Queen's Pawn Game: Zukertort Variation,Queen's Pawn Game,3,Zukertort Variation
4,5,True,95,Mate,White,1,30+3,30,3,1800,...,1523,1400-1599,adivanov2009,1469,1400-1599,54,Philidor Defense,Philidor Defense,5,Unknown


In [65]:
out_csv = r"D:\ches game\clean_chess_for_powerbi.csv"
out_parquet = r"D:\ches game\clean_chess_for_powerbi.parquet"

clean_data.to_csv(out_csv, index=False)

print("✅ Saved cleaned files:")
print(out_csv)


✅ Saved cleaned files:
D:\ches game\clean_chess_for_powerbi.csv
